<a href="https://colab.research.google.com/github/srichallla/Analyzing_Hotels_Dataset/blob/master/hackerrank_doc_multiclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


dealing with class Imbalance problem. using both oversampling followed by undersampling



In [ ]:
import nltk
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.model_selection import train_test_split
#pkgs to deal with class imbalances
from imblearn.combine import SMOTEENN, SMOTETomek

#The first line in input file contains "5485" adding "skiprows" skips 1st line
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/hackerrank_doc_classification.txt', header=None, skiprows=1)
print(df.head(3))

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


                                                   0
0  1 champion products ch approves stock split ch...
1  2 computer terminal systems cpml completes sal...
2  1 cobanco inc cbco year net shr cts vs dlrs ne...


1) Read text data from CSV file.

2) Check for missing values and impute them if needed.

3) Check for duplicates and drop them if present.

4) Verify if class Imbalance exists.

5) Pre-process the data i.e convert to lowercase, remove special characters, digits, expand contarction, stemming, lemmetization, remove stop words etc...

6) Tokenize sentence i.e. break sentences into words.

7) Convert each tokenized sentence to a numerical vectors. Result will be a matrix of numbers, where each row corresponds to one sentence.

8) Deal with class imbalances may be by using oversampling followed by undersampling or other techniques.

9) Split the numerical matrix to Train and Test sets.

10) Train the model.

11) Use the trained model to classify unseen documents.

12) Evaluate the model.



In [ ]:
sentences=[]
labels=[]
#Iterate through all rows 0th column and extract lables and sentences
#If maxsplit is specified, the list will have the maximum of maxsplit+1 items, here each line is split into 2 items at 0->label at 1->whole sentence.
for line in df.iloc[:,0]:
    line=line.split(" ", maxsplit=1)
    labels.append(line[0])
    sentences.append(line[1])

sentences=pd.DataFrame(sentences)
labels=pd.DataFrame(labels)
df2=pd.concat([sentences,labels],axis=1) # Here column names will be '0', so giving names to columns 
df2.columns=["Sentence","Label"]

#shuffling the order of the DataFrame's rows, so that all target labels are mixed
#frac=1 means return all rows (in random order).
#df2 = df2.sample(frac=1).reset_index(drop=True)
print(df2.head(3))
print("\n Count of missing values in each column: \n", df2.isnull().sum()) # No missing values
df2.describe(include='all')

                                            Sentence Label
0  champion products ch approves stock split cham...     1
1  computer terminal systems cpml completes sale ...     2
2  cobanco inc cbco year net shr cts vs dlrs net ...     1

 Count of missing values in each column: 
 Sentence    0
Label       0
dtype: int64


,Sentence,Label
count,5485,5485
unique,5423,8
top,sumita says bank will intervene if necessary b...,1
freq,3,2840


In [ ]:
#Handling duplicate values in the dataframe:
print("Number of duplicates: ", sum(df2.duplicated()))
#To see which rows are duplicated:
df2.loc[df2.duplicated(keep=False),]
#Drop duplicates except for first one
df3=df2.drop_duplicates(keep='first')
print("Number of duplicates after dropping duplicates: ", sum(df3.duplicated()))
print("Dimenstions after removing duplicates: ",df3.shape)
#https://github.com/irfanelahi-ds/document-classification-python/blob/master/document_classification_python_sklearn_nltk.ipynb
print("Before: Class compositions percentage: \n",df3["Label"].value_counts(normalize=True))
print("Before: Class labes distribution counts: \n",df3["Label"].value_counts())
#Looks like we need to deal with class imbalances. Undersampling the majority classe(s) 1 to 400 & 2 to 400, also I will use over-sampling
#minority classes using SMOTE 5:100. These techniques wont work on text data until they are transformed to numbers

Number of duplicates:  58
Number of duplicates after dropping duplicates:  0
Dimenstions after removing duplicates:  (5427, 2)
Before: Class compositions percentage: 
 1    0.522388
2    0.293164
6    0.045145
3    0.044039
8    0.035931
7    0.032430
4    0.019348
5    0.007555
Name: Label, dtype: float64
Before: Class labes distribution counts: 
 1    2835
2    1591
6     245
3     239
8     195
7     176
4     105
5      41
Name: Label, dtype: int64


**Data Pre-processing and Tokenization**

In [ ]:
#Download stop words
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS
print(len(STOP_WORDS))
#Add custom stopwords to existing list, which are more common in traning data
STOP_WORDS.add("reuter")
print("after : " ,len(STOP_WORDS))

326
after :  327


In [ ]:

corpus=df3['Sentence'] #Conatins 5427 sentences
# Data Pre-processing. Converting to lower case, Remove special characters, tokenize(split sentence into words) and remove stop words
def clean_corpus(corpus):
    corpus_cleaned = []
    # normalize each Sentence/line in the corpus
    for sentence in corpus:
        #Covert to lower case
        sentence = sentence.lower()        
        #Remove special characters
        regex_pattern = re.compile(r'[\,+\:\?\!\"\(\)!\'\.\%\[\]]+')
        sentence = regex_pattern.sub(r' ', sentence)        
        sentence = re.sub('[^a-zA-z\s]', '', sentence)
        #tokenize Sentences
        tokenizer = ToktokTokenizer()
        tokens = tokenizer.tokenize(sentence)        
        #Remove stop words
        tokens = [token.strip() for token in tokens]        
        filtered_tokens = [token for token in tokens if token not in STOP_WORDS]  #Comma separated List of strings of one sentence  ['champion', 'products', 'ch',..]    
        filtered_text = ' '.join(filtered_tokens) # line o/p's: champion products ch .... 
        #Truncate sentense to max 500 characters
        filtered_text = filtered_text[0:499]       
        corpus_cleaned.append(filtered_text)   
    return  corpus_cleaned
full_corpus = clean_corpus(corpus)
#clean_corpus(corpus)
print("We have %d lines/Sentences in the corpus." %len(full_corpus))
full_corpus[:10]

We have 5427 lines/Sentences in the corpus.


['champion products ch approves stock split champion products inc said board directors approved stock split common shares shareholders record april company said board voted recommend shareholders annual meeting april increase authorized capital stock mln mln shares',
 'computer terminal systems cpml completes sale computer terminal systems inc said completed sale shares common stock warrants acquire additional mln shares sedio n v lugano switzerland dlrs company said warrants exercisable years purchase price dlrs share computer terminal said sedio right buy additional shares increase total holdings pct computer terminal s outstanding common stock certain circumstances involving change control company company said conditions occur warrants exercisable price eq',
 'cobanco inc cbco year net shr cts vs dlrs net vs assets mln vs mln deposits mln vs mln loans mln vs mln note th qtr available year includes extraordinary gain tax carry forward dlrs cts shr',
 'international inc nd qtr jan ope

**Convert text to matrix of numbers**

In [ ]:
# Initializing TFIDF vectorizer to convert the raw corpus to a matrix of TFIDF features.
vectorizer = TfidfVectorizer(min_df=1,  # min count for relevant vocabulary
    max_features=12000,  # maximum number of features
    strip_accents='unicode',  # replace all accented unicode char 
    # by their corresponding  ASCII char
    analyzer='word',  # features made of words
    token_pattern=r'\w{4,}',  # tokenize only words of 4+ chars
    ngram_range=(1, 1),  # features made of a single tokens
    #use_idf=True,  # enable inverse-document-frequency reweighting
    #smooth_idf=True  # prevents zero division for unseen words
    )
# Creating TFIDF features sparse matrix by fitting it on the specified corpus.
#tfidf_matrix=vectorizer.fit_transform(full_corpus).todense()
tfidf_matrix=vectorizer.fit_transform(full_corpus)
# Grabbing the name of the features.
tfidf_names=vectorizer.get_feature_names()
print("Number of TFIDF Features: %d"%len(tfidf_names))
print("Matrix shape = ",tfidf_matrix.shape)
print("Feature Names:")
print(tfidf_names[:100])
# There are 19657 columns that will be used for training the classifier
#deal with class imbalances using oversampling followed by undersampling
#smote_enn = SMOTEENN(random_state=0)
smt = SMOTETomek(ratio='auto',random_state=0)
X_balanced, y_balanced = smt.fit_resample(tfidf_matrix, df3["Label"])
X_balanced = X_balanced.todense()
#splitting the data into random training and test sets for both independent variables and labels.
X_train_corpus, X_test_corpus, y_labels_train, y_labels_test = train_test_split(X_balanced, y_balanced, test_size=.2)
#analyzing the shape of the training and test data-set:
print('Shape of Training Data: '+str(X_train_corpus.shape))
print('Shape of Test Data: '+str(X_test_corpus.shape))

Number of TFIDF Features: 12000
Matrix shape =  (5427, 12000)
Feature Names:
['aachener', 'aagiy', 'aaix', 'aancor', 'aare', 'aarn', 'aarnoud', 'aaron', 'aati', 'abal', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abatement', 'abbas', 'abbett', 'abbey', 'abbreviated', 'abdel', 'abdelaziz', 'abdul', 'abdulaziz', 'abdullah', 'abev', 'abex', 'abide', 'abiding', 'abidjan', 'abilene', 'abilities', 'ability', 'ablaze', 'able', 'abnn', 'abnormally', 'aboard', 'abolish', 'abolished', 'abolishing', 'aborted', 'abover', 'abroad', 'abrupt', 'absb', 'absence', 'absent', 'absolute', 'absolutely', 'absolve', 'absorb', 'abundant', 'academics', 'academy', 'acccounts', 'accelerate', 'accelerated', 'accelerating', 'acceleration', 'accentuating', 'accept', 'acceptable', 'acceptance', 'acceptances', 'accepted', 'accepting', 'accepts', 'access', 'accession', 'accessories', 'accident', 'accidents', 'accommodate', 'accompanied', 'accompany', 'accompanying', 'accomplishments', 'accord', 'accordance',

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Shape of Training Data: (18099, 12000)
Shape of Test Data: (4525, 12000)


**Training The Model**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mn_bayes=MultinomialNB()
mn_bayes_fit=mn_bayes.fit(X_train_corpus,y_labels_train)

In [ ]:
"""test_corpus =["8 south korean won fixed at month high the bank of korea said it fixed the midrate of the won at to the dollar its highest level since february when it was the won was set at yesterday the won has risen pct against the dollar so far this year after rising pct in reuter",
 "6 occidental oxy in big peru heavy oil find occidental petroleum corp said the shiviyacu development well on block ab in the peruvian amazon jungle is producing barrels of degree gravity oil per day from depths of to and to feet the company said it is drilling a new exploration well on the block southeast shiviyacu miles away reuter ",
 "7 tcf banking and savings hikes prime rate tcf banking and savings fa said it is raising its prime rate to pct from pct effective today ",
 "7 royal bank canada ups u s base rate pct to pct ",
 "5 cereals mcas to be unchanged next week monetary compensatory amounts mca s will be unchanged for the week starting april ec commission officials said cereals mca s are plus points for west germany and the netherlands minus two points for denmark minus eight points for france minus nine for ireland minus for italy minus for britain minus for greece and minus for spain reuter ",
 "5 report on export markets for u s grain delayed the u s agriculture department s report on export markets for u s grain scheudled for release today has been delayed until wednesday april a department spokeswoman said no reason was given for the delay in releasing the monthly report reuter ",
 "4 b and c reorganises commercial operations british and commonwealth shipping co plc bcom l said that it would reorganise its commercial and service operations into a single public grouping with autonomous management the group has expanded rapidly in the past year through the mln stg acquisition of exco international plc and mln bid for steel brothers holdings plc it noted that its operations were now divided between financial services including money broking investment management and forfaiting and more traditional areas such as aviation hotels commodity trading and office equipment it said that each sector had exciting prospects but required different methods of management and financing b and c planned to form a new public company to hold the commercial operations and envisaged it operating with a capital of between mln and mln stg it has retained barclays de zoete wedd to advise on the introduction of independent investors to subscribe for additional capital and believes that the proportion of equity capital held by outside investors would not exceed pct of the total the statement said that with the continued support of b and c together with outside capital the new grouping would emerge as a major group in its own right with the ability to take advantages of opportunities as they arose however the group would not seek a listing for the time being b and c also said that its chairman lord cayzer planned to retire in june the company proposed that he be appointed life president and that current chief executive john gunn should take over as chairman b and c shares eased p to p at gmt reuter ",
 "4 biffex looking to join new futures exchange the baltic international freight futures exchange biffex said it agreed to pursue negotiations with other futures markets on the baltic exchange with a view to merging into a new futures exchange legal advisers have already been instructed to implement amalgamation of the london potato futures association the soya bean meal futures exchange and the london meat futures exchange the london grain futures market has also discussed merging with the other markets the aim of the merger is to seek recognised investment exchange status as required by the financial services act reuter",
 "6 next few months crucial for oil hernandez energy and mines minister arturo hernandez grisanti today told a meeting of regional oil exporters the next few months will be critical to efforts to achieve price recovery and stabilize the market hernandez said while opec and non opec nations have already made some strides in their efforts to strengthen the market the danger of a reversal is always present march and the next two or three months will be a really critical period hernandez said he said we will be able to define a movement either towards market stability and price recovery or depending on the market a reversal earlier this week hernandez said venezuela s oil price has averaged just above dlrs a barrel for the year to date if opec achieves its stated goal of an dlrs a barrel average price he said venezuela s should move up to dlrs hernandez spoke today at the opening of the fifth ministerial meeting of the informal group of latin american and caribbean oil exporters formed in ministers from member states ecuador mexico trinidad tobago and venezuela are attending the two day conference while colombia is present for the first time as an observer hernandez defined the meeting as an informal exchange of ideas about the oil market however the members will also discuss ways to combat proposals for a tax on imported oil currently before the u s congress following the opening session the group of ministers met with president jaime lusinchi at miraflores the presidential palace the delegations to the conference are headed by hernandez of venezuela energy minister javier espinosa of ecuador energy minister kelvin ramnath of trinidad tobago jose luis alcudiai assistant energy secretary of mexico and energy minister guilermno perry rubio of colombia reuter ",
 "1 This is a document ",
 "1 this is another document ",
 "8 documents are seperated by newlines "]"""
"""clean_test_corpus = clean_corpus(X_test_corpus)
print("Cleaned Test corpus:\n" ,clean_test_corpus)
#Transforming sentence from words to numbers
tv_test_features = vectorizer.transform(clean_test_corpus)
print("Test features shape: ",tv_test_features.shape)"""

prediction_mn=mn_bayes_fit.predict(X_test_corpus)
print("Predicted Document Lables are : \n",prediction_mn) #Should output 8, but o/p is 1
"""print("\nCount of non-zeros elements in tfidf_matrix: ",np.count_nonzero(tfidf_matrix))
#The corresponding non-zero values in the array can be obtained with
print("tfidf_matrix dimenstions: ", tfidf_matrix.ndim)
sizeoftext = np.array([len(e) for e in train_corpus])
print("min length (i.e num of characters including spaces) of sentence in train corpus = %d max length of sentence = %d "%(sizeoftext.min(),sizeoftext.max()))
#There are 610 sentences whose size is above 1000 and 157 sentences whose size is above 2000
print("tfidf_matrix[1] index dimenstions: ", tfidf_matrix[1].shape)
print("non-zero elments in tfidf_matrix[1]: ")
tfidf_matrix[np.nonzero(tfidf_matrix[1])]"""

Predicted Document Lables are : 
 ['4' '5' '1' ... '2' '2' '2']


'print("\nCount of non-zeros elements in tfidf_matrix: ",np.count_nonzero(tfidf_matrix))\n#The corresponding non-zero values in the array can be obtained with\nprint("tfidf_matrix dimenstions: ", tfidf_matrix.ndim)\nsizeoftext = np.array([len(e) for e in train_corpus])\nprint("min length (i.e num of characters including spaces) of sentence in train corpus = %d max length of sentence = %d "%(sizeoftext.min(),sizeoftext.max()))\n#There are 610 sentences whose size is above 1000 and 157 sentences whose size is above 2000\nprint("tfidf_matrix[1] index dimenstions: ", tfidf_matrix[1].shape)\nprint("non-zero elments in tfidf_matrix[1]: ")\ntfidf_matrix[np.nonzero(tfidf_matrix[1])]'

with 500 sentence lenght it gives 16492 features.
with 300 sentence lenght it gives 14792 features.
with no sentence truvcation it gives 19,000 features

**Others Promising Evenues to Pursue:**
1) See if you can delete columns/features from "vectorizer.get_feature_names()" if feature name is not present in the corpus vocabulary Or cleaned text document
2) Dealing with class imbalance, stratified smapling etc.
3) Randomly Shuffle rows, to ensure same class/traget values are not together (Doing this did not imporve acccuracy)

**To deal with Imabalanced datasets, I will be doing below:**
1) Under-sampling majority calsse(s) "1" to 300 & "2" 250 using Cluster Centroids (from imblearn.under_sampling import ClusterCentroids)
https://www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets
2) Over-sampling Minority class 5 to 100 using SMOTE (from imblearn.over_sampling import SMOTE)

**Performance metric**

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
#labels_test = ['8', '6', '7' ,'7', '5', '5', '4', '4', '6' ,'1', '1', '8']
#Navie Bayes Accuracy Score
nb_ascore = accuracy_score(y_labels_test, prediction_mn)
print("Navie Bayes Accuracy Score: ", nb_ascore)
print("Confusion Matrix of Bernoulli Naive Bayes Classifier output: ")
confusion_matrix(y_labels_test,prediction_mn)
print("Classification Metrics: ")
print(classification_report(y_labels_test,prediction_mn))

Navie Bayes Accuracy Score:  0.9858563535911602
Confusion Matrix of Bernoulli Naive Bayes Classifier output: 
Classification Metrics: 
              precision    recall  f1-score   support

           1       0.99      0.95      0.97       512
           2       0.97      0.97      0.97       598
           3       0.99      1.00      0.99       563
           4       0.99      1.00      1.00       574
           5       1.00      1.00      1.00       549
           6       0.99      0.99      0.99       582
           7       0.97      0.99      0.98       583
           8       0.98      0.98      0.98       564

    accuracy                           0.99      4525
   macro avg       0.99      0.99      0.99      4525
weighted avg       0.99      0.99      0.99      4525

